In [224]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split

In [225]:
import ImageMethods
import DataMethods

from imp import reload

In [226]:
reload(ImageMethods)
reload(DataMethods)

<module 'DataMethods' from '/Users/Moishe/JupyterProjects/TennisBallDetection/DataMethods.py'>

In [ ]:
name, data = DataMethods.collectBallPosData(place, num)
np.savetxt(name,data,fmt="%i")

(540, 960, 3)
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]
[[0 0 0]]


In [192]:
place = "SBCC"
num = "4"

In [209]:
name2, data2 = DataMethods.collectPositiveDataForVid(place, num, distort=False)
np.save(name2,data2)
print(name2)
print(data2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
ball_data/ball+_data_SBCC_4_full.npy
[[[[ 58  70  80]
   [ 58  70  80]
   [ 52  64  74]
   ...
   [ 66  70  79]
   [ 67  75  84]
   [ 77  85  94]]

  [[ 61  73  83]
   [ 65  77  87]
   [ 76  88  98]
   ...
   [ 66  70  79]
   [ 76  84  93]
   [ 82  90  99]]

  [[ 76  95 104]
   [ 86 105 114]
   [ 76 110 116]
   ...
   [ 62  70  79]
   [ 77  85  94]
   [ 87  95 104]]

  ...

  [[ 60  84  94]
   [ 58  82  92]
   [ 46  82  83]
   ...
   [118 210 173]
   [114 209 176]
   [114 209 176]]

  [[ 54  71  82]
   [ 53  70  81]
   [ 51  71  78]
   ...
   [135 210 182]
   [131 207 182]
   [127 203 178]]

  [[ 70  87  98]
   [ 77  94 105]
   [ 80 100 107]
   ...
   [128 203 175]
   [122 198 173]
   [118 194 169]]]


 [[[ 95 129 142]
   [ 81 115 128]
   [ 6

In [164]:
DataMethods.splitTrainingTestingData("-")

data_files:  ['nonball_data/ball-_data_SBCC_1_full.npy', 'nonball_data/ball-_data_LB_2_full.npy', 'nonball_data/ball-_data_LB_3_full.npy', 'nonball_data/ball-_data_SBCC_3_full.npy', 'nonball_data/ball-_data_SBCC_2_full.npy', 'nonball_data/ball-_data_NP_1_full.npy', 'nonball_data/ball-_data_SS_1_full.npy', 'nonball_data/ball-_data_LB_1_full.npy']
full ball data shape:  (255, 32, 32, 3)
full ball data shape:  (555, 32, 32, 3)
full ball data shape:  (220, 32, 32, 3)
full ball data shape:  (355, 32, 32, 3)
full ball data shape:  (100, 32, 32, 3)
full ball data shape:  (255, 32, 32, 3)
full ball data shape:  (1000, 32, 32, 3)
full ball data shape:  (290, 32, 32, 3)


In [165]:
name, data = DataMethods.combineData("-", "train")
np.save(name,data)
name, data = DataMethods.combineData("-", "test")
np.save(name,data)

(2725, 32, 32, 3)
(305, 32, 32, 3)


In [173]:
negative_x_data = np.load("nonball_data/ball-_data_train.npy")
negative_x_data.shape

(2725, 32, 32, 3)

In [189]:
# make sure that the data is BGR
positive_x_data = np.load("ball_data/ball+_data_train.npy")
positive_y_data = DataMethods.createYData(positive_x_data)

negative_x_data = np.load("nonball_data/ball-_data_train.npy")
negative_y_data = DataMethods.createYData(negative_x_data,case="-")

x_data = np.concatenate((positive_x_data,negative_x_data))
y_data = np.concatenate((positive_y_data,negative_y_data))

x_data_norm = x_data.astype("float32")
x_data_norm = x_data_norm/255.0

# X_train, X_test, Y_train, Y_test = train_test_split(x_data_norm, y_data, test_size=0.1, random_state=42)
X_train = x_data_norm
Y_train = y_data
X_test = np.concatenate((np.load("ball_data/ball+_data_test.npy"),np.load("nonball_data/ball-_data_test.npy"))).astype("float32")/255.0
Y_test = np.concatenate((DataMethods.createYData(np.load("ball_data/ball+_data_test.npy")),DataMethods.createYData(np.load("nonball_data/ball-_data_test.npy"),"-")))

In [182]:
# example of a 3-block vgg style architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))

# example output part of the model
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [183]:
history = model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=None, verbose=1)
# evaluate model
_, acc = model.evaluate(X_test, Y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

Epoch 1/5
5390/5390 [==============================] - 20s 4ms/step - loss: 0.2587 - accuracy: 0.9191
Epoch 2/5
5390/5390 [==============================] - 19s 4ms/step - loss: 0.0912 - accuracy: 0.9660
Epoch 3/5
5390/5390 [==============================] - 19s 3ms/step - loss: 0.0500 - accuracy: 0.9826
Epoch 4/5
5390/5390 [==============================] - 21s 4ms/step - loss: 0.0455 - accuracy: 0.9833
Epoch 5/5
5390/5390 [==============================] - 19s 4ms/step - loss: 0.0411 - accuracy: 0.9872
> 99.336


In [191]:
_, acc = model.evaluate(X_test, Y_test, verbose=0)
acc

0.9950494766235352

In [228]:
predictions = model.predict(X_test[5:10])
print(predictions)
print(Y_test[5:10])

[[9.9999845e-01 1.5450568e-06]
 [9.9999225e-01 7.7446002e-06]
 [9.9987578e-01 1.2420108e-04]
 [9.9999881e-01 1.1348794e-06]
 [5.5248761e-01 4.4751236e-01]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [232]:
sbcc_test = np.load("ball_data/ball+_data_SBCC_4_full.npy").astype("float32")/255.0
predictions = model.predict(sbcc_test)
print(predictions)
_, acc = model.evaluate(sbcc_test, DataMethods.createYData(sbcc_test))
print(acc)

[[9.99999881e-01 1.20515651e-07]
 [1.00000000e+00 1.70186731e-09]
 [1.00000000e+00 1.03211715e-08]
 [1.00000000e+00 3.47666820e-08]
 [1.00000000e+00 6.72480471e-09]
 [1.00000000e+00 2.01189843e-09]
 [9.99999881e-01 1.15940992e-07]
 [9.99987960e-01 1.20407267e-05]
 [9.99996901e-01 3.08924405e-06]
 [9.99997497e-01 2.55020473e-06]
 [9.72139120e-01 2.78608575e-02]
 [9.99959230e-01 4.07545413e-05]
 [1.00000000e+00 8.93914898e-09]
 [9.99998450e-01 1.50336007e-06]
 [9.99963522e-01 3.64418192e-05]
 [9.99987602e-01 1.24531471e-05]
 [9.98650253e-01 1.34980644e-03]
 [9.98556435e-01 1.44351250e-03]
 [9.99930978e-01 6.90090747e-05]
 [9.99991298e-01 8.65733455e-06]
 [9.99973893e-01 2.60830893e-05]
 [9.99991179e-01 8.80699463e-06]
 [9.97371078e-01 2.62889196e-03]
 [9.99988079e-01 1.19249316e-05]
 [9.99993324e-01 6.73319619e-06]
 [9.99987483e-01 1.25030429e-05]
 [9.99137998e-01 8.62030953e-04]
 [9.99484539e-01 5.15487045e-04]
 [9.99711335e-01 2.88610288e-04]
 [9.99931574e-01 6.84247570e-05]
 [9.999451

In [231]:
model.save_weights("./model/weights")